In [ ]:
import cv2
from ultralytics import YOLO
import pytesseract
from pytesseract import Output

# Load YOLOv8 models for vehicle detection and number plate detection
vehicle_model = YOLO('yolov8n.pt')  # Replace with your vehicle detection model
plate_model = YOLO('number_plate_model.pt')  # Replace with your number plate detection model


# Path to your video file
video_path = 'input/output_video.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()



In [ ]:

# Process each frame of the video
while cap.isOpened():
    # Skip 9 out of every 10 frames to speed up process. 
    i=10
    while (i > 0):
        i = i - 1
        ret, frame = cap.read()
        if not ret:
            break

    # Perform vehicle detection on the frame
    vehicle_results = vehicle_model(frame)

    # Ensure results is not empty and contains the boxes attribute
    if vehicle_results and hasattr(vehicle_results[0], 'boxes'):
        # Iterate over detected vehicles
        for vehicle in vehicle_results[0].boxes:
            # Extract bounding box coordinates for vehicle
            x1, y1, x2, y2 = map(int, vehicle.xyxy[0])

            # Crop the detected vehicle region
            vehicle_region = frame[y1:y2, x1:x2]

            # Perform number plate detection on the vehicle region
            plate_results = plate_model(vehicle_region)

            if plate_results and hasattr(plate_results[0], 'boxes'):
                for plate in plate_results[0].boxes:
                    # Extract bounding box coordinates for number plate
                    px1, py1, px2, py2 = map(int, plate.xyxy[0])

                    # Adjust the coordinates relative to the original frame
                    px1 += x1
                    py1 += y1
                    px2 += x1
                    py2 += y1

                    # Crop the detected number plate region
                    number_plate = frame[py1:py2, px1:px2]

                    # Convert the cropped region to grayscale (if needed)
                    gray_plate = cv2.cvtColor(number_plate, cv2.COLOR_BGR2GRAY)

                    # Perform OCR on the cropped number plate
                    text = pytesseract.image_to_string(gray_plate, config='--psm 8')

                    # Draw the bounding box and the detected text on the frame
                    cv2.rectangle(frame, (px1, py1), (px2, py2), (0, 255, 0), 2)
                    cv2.putText(frame, text, (px1, py1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                    print(f'Detected Number Plate Text: {text.strip()}')

            # Draw bounding box around the vehicle
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Display the frame with detections
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
